In [ ]:
# TODO: requirements.txt
# pip install binreader
# pip install matplotlib
# pip install numpy
# pip install nbconvert

In [ ]:
from process import *
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
from process import *

performance: Performance = Performance(load_notes("data/asdf.notes"))

human_fours = performance.human_fours
print([sum(len(measure.notes) for measure in four) for four in human_fours])

In [ ]:

fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)

dist1 = [note.note for four in performance.human_fours for measure in four for note in measure.notes]
dist2 = [note.note for four in performance.agent_fours for measure in four for note in measure.notes]

# We can set the number of bins with the *bins* keyword argument.
axs[0].hist(dist1, bins=10)
axs[1].hist(dist2, bins=10)

plt.show()